# LangChain SQL Agent for Massive Documents Interaction
Tutorial to understand Langchain SQL Agent. (inspired by https://pub.towardsai.net/langchain-sql-agent-for-massive-documents-interaction-510fc4bc65a4)

In [2]:
%pip install pypdf
%pip install langchain
%pip install langchain_openai
%pip install sqlite3


  Using cached pypdf-4.1.0-py3-none-any.whl.metadata (7.4 kB)
   -------------------------------------- 286.1/286.1 kB 490.8 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
  Using cached langchain-0.1.12-py3-none-any.whl.metadata (13 kB)
  Using cached PyYAML-6.0.1-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
     ---------------------------------------- 85.1/85.1 kB 2.4 MB/s eta 0:00:00
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached tenacity-8.2.3-py3-none-any.whl.metadata (1.0 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached charset_normalizer-3.3.2-cp311-cp311-win_amd64.whl.metadata (34 kB)
  Using cached anyio-4.3.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
     ---------------------------------------- 50.7/50.7 kB 1.3 MB/s eta 0:00:00
  Using cached annotated_types-0.6.0-py3-none-any.whl.metadata (12 kB)
  Using cached pyda

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dvc 3.15.2 requires rich>=12, which is not installed.


  Using cached langchain_openai-0.0.8-py3-none-any.whl.metadata (2.5 kB)
  Using cached tiktoken-0.6.0-cp311-cp311-win_amd64.whl.metadata (6.8 kB)
  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached regex-2023.12.25-cp311-cp311-win_amd64.whl.metadata (41 kB)
  Using cached httpcore-1.0.4-py3-none-any.whl.metadata (20 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
   -------------------------------------- 257.5/257.5 kB 150.7 kB/s eta 0:00:00
Using cached tiktoken-0.6.0-cp311-cp311-win_amd64.whl (798 kB)
Using cached httpx-0.27.0-py3-none-any.whl (75 kB)
Using cached httpcore-1.0.4-py3-none-any.whl (77 kB)
Using cached regex-2023.12.25-cp311-cp311-win_amd64.whl (269 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [17]:
%pip install faiss-cpu

   ---------------------------------------- 14.5/14.5 MB 26.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install cffi


  Using cached cffi-1.16.0-cp311-cp311-win_amd64.whl.metadata (1.5 kB)
Using cached cffi-1.16.0-cp311-cp311-win_amd64.whl (181 kB)
Using cached pycparser-2.21-py2.py3-none-any.whl (118 kB)
Note: you may need to restart the kernel to use updated packages.


In [10]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [1]:
from dotenv import load_dotenv
import os

load_dotenv()


True

In [2]:
from pypdf import PdfReader

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

## Analyze Documents using Python, LangChain and GPT

In [3]:
document_content = None

def visitor_body(text, cm, tm, fontDict, fontSize):
    y = tm[5]
    if text and 35 < y < 770:
        page_contents.append(text)

with open(f'../docs/ZMP_55852_XBO_1000_W_HS_OFR.pdf', 'rb') as file:
    pdf_reader = PdfReader(file)

    page_contents = []

    for page in pdf_reader.pages:
        page.extract_text(visitor_text=visitor_body)

    document_content = "\n".join(page_contents)

print(document_content)

Product family benefits
_
Short arc with very high luminance for brighter screen illumination
_
Constant color temperature of 6,000 K throughout the entire lamp lifetime
_
Easy to maintain
_
High arc stability
_
Instant light on screen thanks to hot restart function
_
Wide dimming range
Product family features
_
Color temperature: approx. 6,000 K (Daylight)
_
Wattage: 450…10,000 W
_
Very good color rendering index: Ra >
Product datasheet


 
XBO 1000 W/HS OFR
XBO for cinema projection | Xenon short-arc lamps 450…10,000 W


 
Areas of application
_
Classic 35 mm film projection
_
Digital film and video projection
_
Architectural and effect light (“Light Finger”)
_
Sunlight simulation
__


Technical data
Electrical data
Nominal current
50 A
Current control range
30…55 A
Nominal wattage
1000.00 W
Nominal voltage
19.0 V
Dimensions & weight


 
Diameter
40.0 mm
Length
235.0 mm
Length with base excl. base pins/connection
205.00 mm
Light center length (LCL)
95.0 mm 
1)
Electrode gap cold
3.6 

In [4]:
system_message = """
You analyze product descriptions to export them into a JSON format. I will present you with a product data sheet and describe the individual JSON objects and properties with <<<. You then create a JSON object from another product data sheet.

>>> Example product:

Product family benefits <<< benefits (string[])
_
Short arc with very high luminance for brighter screen illumination <<< benefits.[*]
_
Constant color temperature of 6,000 K throughout the entire lamp lifetime <<< benefits.[*]

[..]

_
Wide dimming range <<< benefits.[*]
Product family features <<< product_family (object)
_
Color temperature: approx. 6,000 K (Daylight) <<< product_family.temperature = 6000
_
Wattage: 450…10,000 W <<< product_family.watts_min = 450, product_family.watts_max = 10000
_
Very good color rendering index: Ra >
Product datasheet


 
XBO 1000 W/HS OFR <<< name
XBO for cinema projection | Xenon short-arc lamps 450…10,000 W <<< description

[..]

Technical data
Electrical data <<< technical_data (object)
Nominal current
50 A <<< technical_data.nominal_current = 50.00
Current control range
30…55 A <<< technical_data.control_range = 30, technical_data.control_range = 55
Nominal wattage
1000.00 W <<< technical_data.nominal_wattage = 1000.00
Nominal voltage
19.0 V <<< technical_data.nominal_voltage = 19.0
Dimensions & weight <<< dimensions (object)

[..]

Safe Use Instruction
The identification of the Candidate List substance is <<< environmental_information.safe_use (beginning of string)

sufficient to allow safe use of the article. <<< environmental_information.safe_use (end of string)
Declaration No. in SCIP database
22b5c075-11fc-41b0-ad60-dec034d8f30c <<< environmental_information.scip_declaration_number (single string!)
Country specific information

[..]

Shipping carton box

1
410 mm x 184 mm x <<< packaging_unity.length = 410, packaging_unit.width = 184

180 mm <<< packaging_unit.height = 180

[..]
"""

In [8]:
chat = ChatOpenAI(model_name='gpt-3.5-turbo-0125', temperature=0, openai_api_key=os.getenv('OPENAI_API_KEY'))

def convert_to_json(document_content):
    messages = [
        SystemMessage(
            content=system_message
        ),
        HumanMessage(
            content=document_content
        )
    ]

    answer = chat.invoke(messages)
    return answer.content

json = convert_to_json(document_content)

print(json)

{
  "name": "XBO 1000 W/HS OFR",
  "description": "XBO for cinema projection | Xenon short-arc lamps 450…10,000 W",
  "areas_of_application": [
    "Classic 35 mm film projection",
    "Digital film and video projection",
    "Architectural and effect light (“Light Finger”)",
    "Sunlight simulation"
  ],
  "product_family": {
    "benefits": [
      "Short arc with very high luminance for brighter screen illumination",
      "Constant color temperature of 6,000 K throughout the entire lamp lifetime",
      "Easy to maintain",
      "High arc stability",
      "Instant light on screen thanks to hot restart function",
      "Wide dimming range"
    ],
    "features": {
      "temperature": 6000,
      "watts_min": 450,
      "watts_max": 10000,
      "color_rendering_index": "Very good"
    }
  },
  "technical_data": {
    "electrical_data": {
      "nominal_current": 50.00,
      "control_range": [30, 55],
      "nominal_wattage": 1000.00,
      "nominal_voltage": 19.0
    },
    "dim

In [9]:
## We add JSON Schema to the prompt to improve the results
system_message = """
    You analyze product descriptions to export them into a JSON format. I will present you with a product data sheet and describe the individual JSON objects and properties with <<<. You then create a JSON object from another product data sheet.

>>> Example product:

Product family benefits <<< benefits (string[])

[..]

-----

Provide your JSON in the following schema:

{
  "type": "object",
  "properties": {
    "name": {
      "type": "string"
    },
    "description": {
      "type": "string"
    },
    "applications": {
      "type": "array",
      "items": {
        "type": "string"
      }
    },
    "benefits": {
      "type": "array",
      "items": {
        "type": "string"
      }
    },
    "product_family": {
      "type": "object",
      "properties": {
        "temperature": {
          "type": "number"
        },
        "watts_min": {
          "type": "number"
        },
        "watts_max": {
          "type": "number"
        }
      }
    },
    "technical_data": {
      "type": "object",
      "properties": {
        "nominal_current": {
          "type": "number"
        },
        "control_range_min": {
          "type": "number"
        },
        "control_range_max": {
          "type": "number"
        },
        "nominal_wattage": {
          "type": "number"
        },
        "nominal_voltage": {
          "type": "number"
        }
      }
    },
    "dimensions": {
      "type": "object",
      "properties": {
        "diameter": {
          "type": "number"
        },
        "length": {
          "type": "number"
        },
        "length_base": {
          "type": "number"
        },
        "light_center_length": {
          "type": "number"
        },
        "electrode_gap": {
          "type": "number"
        },
        "weight": {
          "type": "number"
        }
      }
    },
    "operating_conditions": {
      "type": "object",
      "properties": {
        "max_temp": {
          "type": "string"
        },
        "lifespan": {
          "type": "number"
        },
        "service_lifetime": {
          "type": "number"
        }
      }
    },
    "logistical_data": {
      "type": "object",
      "properties": {
        "product_code": {
          "type": "string"
        },
        "product_name": {
          "type": "string"
        },
        "packaging_unit": {
          "type": "object",
          "properties": {
            "product_code": {
              "type": "string"
            },
            "product_name": {
              "type": "string"
            },
            "length": {
              "type": "number"
            },
            "width": {
              "type": "number"
            },
            "height": {
              "type": "number"
            },
            "volume": {
              "type": "number"
            },
            "weight": {
              "type": "number"
            }
          }
        }
      }
    }
  }
}
    """

In [10]:
chat = ChatOpenAI(model_name='gpt-3.5-turbo-0125', temperature=0, openai_api_key=os.getenv('OPENAI_API_KEY'))

def convert_to_json(document_content):
    messages = [
        SystemMessage(
            content=system_message
        ),
        HumanMessage(
            content=document_content
        )
    ]

    answer = chat.invoke(messages)
    return answer.content

json = convert_to_json(document_content)

print(json)

{
  "name": "XBO 1000 W/HS OFR",
  "description": "XBO for cinema projection | Xenon short-arc lamps 450…10,000 W",
  "applications": [
    "Classic 35 mm film projection",
    "Digital film and video projection",
    "Architectural and effect light (“Light Finger”)",
    "Sunlight simulation"
  ],
  "benefits": [
    "Short arc with very high luminance for brighter screen illumination",
    "Constant color temperature of 6,000 K throughout the entire lamp lifetime",
    "Easy to maintain",
    "High arc stability",
    "Instant light on screen thanks to hot restart function",
    "Wide dimming range"
  ],
  "product_family": {
    "temperature": 6000,
    "watts_min": 450,
    "watts_max": 10000
  },
  "technical_data": {
    "nominal_current": 50,
    "control_range_min": 30,
    "control_range_max": 55,
    "nominal_wattage": 1000,
    "nominal_voltage": 19
  },
  "dimensions": {
    "diameter": 40,
    "length": 235,
    "length_base": 205,
    "light_center_length": 95,
    "elect

## Q&A

In [11]:
import os

from langchain.chains.question_answering import load_qa_chain
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [14]:
pdf_files = [f for f in os.listdir('../docs') if f.endswith('.pdf')]

json_documents = []

for pdf_file in pdf_files:
    with open(f'../docs/{pdf_file}', 'rb') as file:
        pdf_reader = PdfReader(file)

        page_contents = []

        for page in pdf_reader.pages:
            page.extract_text(visitor_text=visitor_body)

        json = convert_to_json("\n".join(page_contents))

        json_documents.append(json)

In [18]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

docsearch = FAISS.from_texts(json_documents, embeddings)

In [19]:
chain = load_qa_chain(chat, chain_type="stuff", verbose=True)

query = "Can I fit the XBO 1000 W/HS OFR into a box with 350mm length and 200mm width?"

docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

c:\Users\colektia\anaconda3\envs\client-iq\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{
  "name": "XBO 1000 W/HS OFR",
  "description": "XBO for cinema projection | Xenon short-arc lamps 450…10,000 W",
  "applications": [
    "Classic 35 mm film projection",
    "Digital film and video projection",
    "Architectural and effect light (“Light Finger”)",
    "Sunlight simulation"
  ],
  "benefits": [
    "Short arc with very high luminance for brighter screen illumination",
    "Constant color temperature of 6,000 K throughout the entire lamp lifetime",
    "Easy to maintain",
    "High arc stability",
    "Instant light on screen thanks to hot restart function",
    "Wide dimming range"
  ],
  "product_family": {
    "temperature": 6000,
    "watts_min": 450,
    "watts_max": 10000
  }

'The packaging dimensions for the XBO 1000 W/HS OFR are 410mm in length and 184mm in width. Therefore, it would not fit into a box with 350mm length and 200mm width.'

## SQLite Database Creation


#### Optional: Defining output classes

In [21]:
import json

from typing import Any, List, Optional
from dataclasses import dataclass, field

In [52]:
@dataclass
class ProductFamily:
    watts_min: Optional[int] = field(default=0)
    watts_max: Optional[int] = field(default=0)
    temperature: Optional[int] = field(default=0)

    @staticmethod
    def from_dict(obj: Any) -> 'ProductFamily':
        _watts_min = int(obj.get("watts_min"))
        _watts_max = int(obj.get("watts_max"))
        _temperature = obj.get("temperature")
        return ProductFamily(_watts_min, _watts_max, _temperature)

@dataclass
class Dimensions:
    diameter: Optional[int] = field(default=0)
    length: Optional[int] = field(default=0)
    weight: Optional[int] = field(default=0)

    @staticmethod
    def from_dict(obj: Any) -> 'Dimensions':
        _diameter = int(obj.get("diameter"))
        _length = int(obj.get("length"))
        _weight = int(obj.get("weight"))
        return Dimensions(_diameter, _length, _weight)

@dataclass
class PackagingUnit:
    length: Optional[int] = field(default=0)
    width: Optional[int] = field(default=0)
    height: Optional[int] = field(default=0)
    weight: Optional[int] = field(default=0)

    @staticmethod
    def from_dict(obj: Any) -> 'PackagingUnit':
        _length = int(obj.get("length"))
        _width = int(obj.get("width"))
        _height = int(obj.get("height"))
        _weight = int(obj.get("weight"))
        return PackagingUnit(_length, _width, _height, _weight)

@dataclass
class LogisticalData:
    packaging_unit: PackagingUnit

    @staticmethod
    def from_dict(obj: Any) -> 'LogisticalData':
        _packaging_unit = PackagingUnit.from_dict(obj.get("packaging_unit"))
        return LogisticalData(_packaging_unit)

@dataclass
class Product:
    name: str
    description: str
    benefits: List[str]
    product_family: ProductFamily
    dimensions: Dimensions
    logistical_data: LogisticalData
    applications: List[str] = field(default_factory=list)

    @staticmethod
    def from_dict(obj: Any) -> 'Product':
        _name = str(obj.get("name"))
        _description = str(obj.get("description"))
        _benefits = obj.get("benefits")
        _product_family = ProductFamily.from_dict(obj.get("product_family"))
        _dimensions = Dimensions.from_dict(obj.get("dimensions"))
        _logistical_data = LogisticalData.from_dict(obj.get("logistical_data"))
        _applications = obj.get("applications", [])
        return Product(_name, _description, _benefits, _product_family, _dimensions, _logistical_data, _applications)


In [23]:
@dataclass
class ProductFamily:
    watts_min: int
    watts_max: int
    temperature: Optional[int] = field(default=0)

    @staticmethod
    def from_dict(obj: Any) -> 'ProductFamily':
        _watts_min = int(obj.get("watts_min"))
        _watts_max = int(obj.get("watts_max"))
        _temperature = obj.get("temperature")
        return ProductFamily(_watts_min, _watts_max, _temperature)

# [..]

@dataclass
class Product:
    name: str
    description: str
    benefits: List[str]
    product_family: ProductFamily

    @staticmethod
    def from_dict(obj: Any) -> 'Product':
        _name = str(obj.get("name"))
        _description = str(obj.get("description"))
        _benefits = obj.get("benefits")
        _product_family = ProductFamily.from_dict(obj.get("product_family"))
        return Product(_name, _description, _benefits, _product_family)

In [53]:
import traceback

pdf_files = [f for f in os.listdir('../docs') if f.endswith('.pdf')]
products = []

for pdf_file in pdf_files:
    json_content = None
    try:
        with open(f'../docs/{pdf_file}', 'rb') as file:
            pdf_reader = PdfReader(file)

            page_contents = []

            for page in pdf_reader.pages:
                page.extract_text(visitor_text=visitor_body)

            document_content = "\n".join(page_contents)

            json_content = convert_to_json(document_content)
            json_data = json.loads(json_content)

            product = Product.from_dict(json_data)
            products.append(product)
    except Exception as e:
        print("{filename} has a problem: {e}".format(filename=pdf_file, e=e))
        print(traceback.format_exc())
        print(json_content)
    else:
        os.rename(f'../docs/{pdf_file}', f'../docs/processed/{pdf_file}')

GPS01_1028553_XBO_HPS_Xenon_short-arc_lamps.pdf has a problem: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'
Traceback (most recent call last):
  File "C:\Users\colektia\AppData\Local\Temp\ipykernel_70568\974335442.py", line 22, in <module>
    product = Product.from_dict(json_data)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\colektia\AppData\Local\Temp\ipykernel_70568\2426096723.py", line 67, in from_dict
    _dimensions = Dimensions.from_dict(obj.get("dimensions"))
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\colektia\AppData\Local\Temp\ipykernel_70568\2426096723.py", line 24, in from_dict
    _weight = int(obj.get("weight"))
              ^^^^^^^^^^^^^^^^^^^^^^
TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

{
  "name": "XBO HPS for digital cinema projection",
  "description": "Xenon short-arc lamps with high luminance for brighter screen illumi

In [54]:
products

[Product(name='XBO DHP for digital cinema projection', description='Xenon short-arc lamps with high luminance for brighter screen illumination, constant 6000 K color temperature, easy maintenance, high arc stability, hot restart, instant light on screen, and wide dimming range.', benefits=['Short arc with highest luminance for brighter screen illumination', 'Lifetime of reliable performances with constant 6000 K color temperature', 'Easy to maintain', 'High level of arc stability', 'Hot restart and instant light on screen', 'Dimmable in a wide range'], product_family=ProductFamily(watts_min=1200, watts_max=7000, temperature=230), dimensions=Dimensions(diameter=78, length=436, weight=1000), logistical_data=LogisticalData(packaging_unit=PackagingUnit(length=170, width=436, height=393, weight=1023)), applications=['Cinema projection']),
 Product(name='XBO HPN for digital cinema projection', description='Xenon short-arc lamps with high luminance for brighter screen illumination, reliable p

In [55]:
import sqlite3


In [56]:
if(os.path.exists('../db') == False):
    os.makedirs('../db')

db_file = '../db/products.db'

db_connection = sqlite3.connect(db_file)
db_cursor = db_connection.cursor()

db_cursor.execute('''CREATE TABLE IF NOT EXISTS Product
    (name TEXT PRIMARY KEY,
    description TEXT,
    temperature INTEGER,
    watts_min INTEGER,
    watts_max INTEGER,
    dimension_diameter REAL,
    dimension_length REAL,
    dimension_weight REAL,
    packaging_length INTEGER,
    packaging_width INTEGER,
    packaging_height INTEGER,
    packaging_weight REAL) WITHOUT ROWID
''')

db_cursor.execute('''
CREATE TABLE IF NOT EXISTS ProductApplication (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    product TEXT,
    text TEXT NOT NULL,
    FOREIGN KEY (product) REFERENCES Product(name)
)
''')

db_cursor.execute('''
CREATE TABLE IF NOT EXISTS ProductBenefit (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    product TEXT,
    text TEXT NOT NULL,
    FOREIGN KEY (product) REFERENCES Product(name)
)
''')

db_connection.commit()

In [57]:
products_sql_tuples = [(
    p.name,
    p.description,
    p.product_family.temperature,
    p.product_family.watts_min,
    p.product_family.watts_max,
    p.dimensions.diameter,
    p.dimensions.length,
    p.dimensions.weight,
    p.logistical_data.packaging_unit.length,
    p.logistical_data.packaging_unit.width,
    p.logistical_data.packaging_unit.height,
    p.logistical_data.packaging_unit.weight,) for p in products]

applications_sql_tuples = []
for product in products:
    applications_sql_tuples.extend([(product.name, application) for application in product.applications])

benefits_sql_tuples = []
for product in products:
    benefits_sql_tuples.extend([(product.name, benefit) for benefit in product.benefits])

In [58]:
db_cursor.executemany('''
    REPLACE INTO Product (name, description, temperature, watts_min, watts_max, dimension_diameter, dimension_length, dimension_weight, packaging_length, packaging_width, packaging_height, packaging_weight)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
''', products_sql_tuples)

db_cursor.executemany('''
    REPLACE INTO ProductApplication (product, text)
    VALUES (?, ?)
''', applications_sql_tuples)

db_cursor.executemany('''
    REPLACE INTO ProductBenefit (product, text)
    VALUES (?, ?)
''', benefits_sql_tuples)

db_connection.commit()

In [59]:
db_cursor.close()
db_connection.close()